# Image Filtering

## Overview

### Learning Objectives

* Becoming familiar with the Pythonic and Object-oriented ITK interfaces
* Understand the image processing pipeline model used in ITK
* Gain familiarity with some of the filtering algorithms available in ITK

## Data and Process Objects

While **data objects** (e.g., [Image's](https://itk.org/Doxygen/html/classitk_1_1Image.html) and [Mesh's](https://itk.org/Doxygen/html/classitk_1_1Mesh.html)) are used to represent data, **process objects** are classes that operate on data objects and may produce new data objects.

Process objects are classed as sources, filter objects, or mappers.

**Sources** (such as readers) produce data, **filter** objects take in data and process it to produce new data, and **mappers** accept data for output either to a file or some other system. 

Sometimes the term *filter* is used broadly to refer to all three types.

![Data pipeline](data/data-pipeline.png)

Typically **data objects** and **process objects** are connected together using the `SetInput()` and `GetOutput()` methods.

Generation of new outputs and pixel data does not occur until the `Update()` method is called on the **end** of the pipeline (on the process object or the data object).

![Pipeline updates](data/pipeline-updates.png)

The **data** associated with multi-dimensional images **is large and becoming larger**. 

Any practical image analysis system must address this fact in order to be useful in applications with multi-dimensional images. ITK addresses this problem via its **data streaming** facility.

![Streaming](data/streaming.gif)

Streaming is performed by splitting the image into non-overlapping **regions** at the **end of the pipeline**. The *RequestedRegion* then propagates up the pipeline.



There are three named [ImageRegion](https://itk.org/Doxygen/html/classitk_1_1ImageRegion.html)'s encountered in ITK:

- *BufferedRegion*: The region of pixels stored in memory

- *LargestPossibleRegion*: The largest possible region of an image

- *RequestedRegion*: The region requested on a single processing pass when streaming. The BufferedRegion and LargestPossibleRegion are as large or larger than the RequestedRegion

For more information, see,

- The **Filtering** chapter of Book 2 of the [ITK Software Guide](https://itk.org/ItkSoftwareGuide.pdf).
- The **Data Processing Pipeline** section of the *System Overview* chapter of Book 1 of the ITK Software Guide.
- The **Data Representation** chapter of Book 1 of the ITK Software Guide.

## Tutorial

In [1]:
import numpy as np
import itk
from itkwidgets import view
from ipywidgets import interactive
import ipywidgets as widgets

We can monitor when a filter gets called by registering a command that gets called when [ProgressEvent's](https://itk.org/Doxygen/html/classitk_1_1EventObject.html) occur.


In [2]:
itk.auto_progress(2)

Start an image processing pipeline with a source, an `ImageFileReader`.

In [7]:
file_name = 'data/PacMan.png'
reader = itk.ImageFileReader.New(FileName=file_name)

Next, let's create a smoothing filter. To connect the pipeline, specify the `Output` of the reader as the `Input` to the smoother.

In [8]:
smoother = itk.RecursiveGaussianImageFilter.New(Input=reader.GetOutput())

At this point, no output images have been generated.

We have configured the simple pipeline:

reader -> smoother

In [9]:
print("reader's Output: %s" % reader.GetOutput())
print("smoother's Output: %s" % smoother.GetOutput())

reader's Output: Image (0x1af0280)
  RTTI typeinfo:   itk::Image<unsigned char, 2u>
  Reference Count: 3
  Modified Time: 395
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x1b0b820) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x3d86a00)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1
      Modified Time: 392
      Debug: Off
      Object Name: 
      Observers: 
        none
      Po

To generate the filter outputs, we must call `Update()` on the filter at the end of the pipeline. In this case, it is the smoother.

In [10]:
smoother.Update()

print("reader's Output: %s" % reader.GetOutput())
print("smoother's Output: %s" % smoother.GetOutput())

reader's Output: Image (0x1af0280)
  RTTI typeinfo:   itk::Image<unsigned char, 2u>
  Reference Count: 3
  Modified Time: 576
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x1b0b820) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 405
  UpdateMTime: 577
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x3d86a00)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1
      Modified Time: 574
      Debug: Off
      Object Name: 
      Observers: 
     

Running itkImageFileReaderIUC2... done
Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


Let's view the images.

In [12]:
image = reader.GetOutput()
view(image, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

In [13]:
smoothed = smoother.GetOutput()
view(smoothed, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

If we call `Update()` on the pipeline, the output pixel data is not needlessly generated because the pipeline is up-to-date.

In [14]:
smoother.Update()

Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


However, if we change the amount of smoothing, new pixel data does need to be generated from the output of the smoother.

In [15]:
smoother.SetSigma(10.0)
smoother.Update()

Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


In [17]:
view(smoothed, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

Note, however, that the reader does generate its output because it is up-to-date and upstream from the smoother.

If we artificially modify the reader, both the reader and the smoother need to regenerate their outputs.

In [18]:
reader.Modified()

smoother.Update()

Running itkImageFileReaderIUC2... done
Running itkRecursiveGaussianImageFilterIUC2IUC2... done
done


### Exercise 1: The effect of Sigma

Change the value of *Sigma* on the smoothing filter.

- How is the output effected?
- What are the units of Sigma?

In [ ]:
# fileName = 'data/PacMan.png'
# reader = itk.ImageFileReader.New(FileName=fileName)
# smoother = itk.RecursiveGaussianImageFilter.New(Input=reader.GetOutput())
# smoother.SetSigma(XX)
# smoother.Update()
# view(smoother.GetOutput())

In [31]:
# %load solutions/3_Image_Filtering_Exercise1.py

### Exercise 2: Does setting a filter parameter to its current value cause regeneration of its output?

Call `smoother.SetSigma(smoother.GetSigma())` then `smoother.Update()`. 

- Is the output regenerated? 
- Is this expected / desirable?

In [36]:
# %load solutions/3_Image_Filtering_Exercise2.py

### Exercise 3: Find other image filtering algorithms

The classes in ITK are organized into **Modules**, and collections of Modules are organized into **Groups**. Examine the [Image Smoothing](https://itk.org/Doxygen/html/group__ITKSmoothing.html) Module and the [Filtering](https://itk.org/Doxygen/html/group__Group-Filtering.html) Group. Can any other smoothing or denoising classes be found? 

In [40]:
# %load solutions/3_Image_Filtering_Exercise3.py

The answer example demonstrates how to combine `ipywidgets` native [interactive](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html) function to quickly create widgets coupled with an [`itkwidgets`](https://github.com/InsightSoftwareConsortium/itkwidgets) viewer. This is an effective method to explore algorithm parameters.

### Enjoy ITK!